<span><img src="lai.png" width="20%" /></span>

# Learning to Predict the Global Risks Interconnections from the Web
.
**Ernesto Diaz-Aviles $<ernesto@libreai.com>$**

Project code-name: **Minerva**

_This project is supported by the [Digital News Innovation Fund (DNI Fund) – Google News Initiative](https://newsinitiative.withgoogle.com/dnifund/)._ 

Project ID: **r3-ywoLejjZpgAv**

<img src="images/minerva_overview.png" width="85%"/>


In [1]:
import sys
sys.path.append('..')

In [2]:
import numpy as np
import pandas as pd
import json

%matplotlib notebook
import matplotlib

from minerva.core import Minerva
from minerva.nowcasting.nnx import NNX
from minerva.nowcasting.nns import NNS
from minerva import toolbox
from collections import defaultdict, Counter


import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

logging.getLogger('ccnews').setLevel(logging.CRITICAL)
logging.getLogger('jieba').setLevel(logging.CRITICAL)

/Users/bluebalam/.pyenv/versions/3.6.5/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/bluebalam/.pyenv/versions/3.6.5/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [3]:
m = Minerva()

In [4]:
m._Minerva__risk_data[['id', 'category', 'long_label', 'description']]

,id,category,long_label,description
0,ASSET,Economic,Asset bubbles in a major economy,Unsustainably overpriced assets such as commod...
1,DEFLATION,Economic,Deflation in a major economy,Prolonged near-zero inflation or deflation in ...
2,FINANCIALMECH,Economic,Failure of a major financial mechanism or inst...,Collapse of a financial institution and/or mal...
3,CRITICALINFRA,Economic,Failure/shortfall of critical infrastructure,"Failure to adequately invest in, upgrade and/o..."
4,FISCALCRISES,Economic,Fiscal crises in key economies,Excessive debt burdens that generate sovereign...
5,UNEMPLOYMENT,Economic,High structural unemployment or underemployment,A sustained high level of unemployment or unde...
6,ILLICITTRADE,Economic,"Illicit trade (e.g. illicit financial flows, t...",Large-scale activities outside the legal frame...
7,ENERGY,Economic,Severe energy price shock (increase or decrease),Significant energy price increases or decrease...
8,INFLATION,Economic,Unmanageable inflation,Unmanageable increases in the general price le...
9,WEATHEREVENTS,Environmental,"Extreme weather events (e.g. floods, storms, e...","Major property, infrastructure and/or environm..."


## Minerva Pipeline

<img src="images/pipeline-overview.png" width="70%" style="text-align: center;" />



We ingest news articles provided by the [Common Crawl](http://commoncrawl.org/) dataset, in particular the [CC-News](http://commoncrawl.org/2016/10/news-dataset-available/) collection.

The CC-News dataset is hosted by Amazon AWS S3 in the commoncrawl bucket at /crawl-data/CC-NEWS/. [WARC](https://en.wikipedia.org/wiki/Web_ARChive) files are released on a daily basis, identifiable by file name prefix which contains year, month and day. A full list of the published WARC files to-date can be obtained with the AWS Command Line Interface and the command:

```bash
$> aws s3 --no-sign-request ls --recursive s3://commoncrawl/crawl-data/CC-NEWS/
```

and to fetch a specific WARC file to the local directory, one could use the following command:

```bash
$> aws s3 --no-sign-request cp s3://commoncrawl/crawl-data/CC-NEWS/2018/08/CC-NEWS-20180801004757-00138.warc.gz ./
```
Please refer to the [CC-News](http://commoncrawl.org/2016/10/news-dataset-available/) information page for more details.

### News Article Extraction
News articles and metadata information can be extracted from the WARC file. Articles that are extracted are in English and published from specified news media outlets.

In [4]:
%%time
%%capture capt
# This method extracts the news articles from a WARC file, in this example 'CC-NEWS-20180801004757-00138.warc.gz' 
# and saves the results in the specified 'out_dir' adding the 'articles_extracted' extension to the name, e.g.,
# this call would extract the articles and save them to a pickle file with the following name:
# 'CC-NEWS-20180801004757-00138.extracted_articles.pkl' in the 'out_dir' specified: '../data/'

# We already run this command for you, it is placed here for reference in case you want to reproduce the results
#m.extract_and_save_articles_from_ccnews(["../data/CC-NEWS-20180801004757-00138.warc.gz"], out_dir="../data/")

# as an example to inspect the format of the aritles extracted, we can load the pickle file:

articles = m.load("../data/CC-NEWS-20180801004757-00138.extracted_articles.pkl")

CPU times: user 12.5 ms, sys: 4.52 ms, total: 17 ms
Wall time: 20.7 ms


the extracted files are stored in a list:

In [16]:
type(articles)

list

and have as type: `NewsArticle`:

In [20]:
# an article:
an_article = articles[0]
type(an_article)

NewsArticle.NewsArticle

information extracted can be accessed as follows:

In [21]:
an_article.get_dict()

{'authors': ['Kyle Madson', 'Mike Patton'],
 'date_download': datetime.datetime(2018, 8, 1, 0, 51, 27, tzinfo=tzutc()),
 'date_modify': None,
 'date_publish': datetime.datetime(2018, 7, 31, 21, 40, 30),
 'description': 'The Patriots flipped the script and signed a Titans player.',
 'filename': 'https%3A%2F%2Ftitanswire.usatoday.com%2F2018%2F07%2F31%2Fpatriots-flip-script-sign-former-titans-ol%2F.json',
 'image_url': 'https://usattitanswire.files.wordpress.com/2017/11/gettyimages-870626694.jpg?w=1024&h=576&crop=1',
 'language': 'en',
 'localpath': None,
 'source_domain': 'titanswire.usatoday.com',
 'text': 'The migration of players between the Patriots and Titans has typically gone north to south over the last couple seasons. This time it’s going the other way. Former Titans offensive lineman Brian Schwenke is heading to New England according to ESPN’s Field Yates.\nThe Patriots are signing veteran OL Brian Schwenke, according to a league source. He spent the 2017 season with the Titans

---

### Global Risk Classification

<img src="images/01-pipeline-global-risk-classification.png" width="80%" style="text-align: center;" />

---
### Entity Extraction

<img src="images/02-pipeline-entity-extraction.png" width="60%" style="text-align: center;" />


In [4]:
%%time
%%capture capt
# Global Risk Classification and Entity Extraction can be performed with the following method: 
# global_risks_infos = m.extract_from_ccnews(["../data/CC-NEWS-20180801004757-00138.warc.gz"], fname_out="../data/global_risks_infos.pkl")
# ---
# since this is already computed for this example, we can simple load the file 'global_risks_infos.pkl':
global_risks_infos = m.load("../data/global_risks_infos.pkl")

CPU times: user 8.42 ms, sys: 2.14 ms, total: 10.6 ms
Wall time: 9.7 ms


$ $


After the method `extract_from_ccnews()` completes the Global Risks information extraction, it returns a list of dictionaries for each article. The information structure is shown in the following example for the first element of the list:

In [5]:
global_risks_infos[0]

{'title': "The new Medicare for All report shouldn't have the left celebrating",
 'source_domain': 'www.washingtonpost.com',
 'url': 'https://www.washingtonpost.com/opinions/have-free-marketers-and-the-far-left-finally-found-something-they-can-agree-on/2018/07/31/6d1436ce-9504-11e8-80e1-00e80e1fdf43_story.html',
 'image_url': 'https://www.washingtonpost.com/resizer/vSQgviM_5aWhwHxNA3bRL5sggZ4=/1484x0/arc-anglerfish-washpost-prod-washpost.s3.amazonaws.com/public/SGM7BOEUIMI6RAIML6TQLET5KQ.jpg',
 'risk': 'STATECOLLAPSE',
 'risk_sim': 0.6614330556727485,
 'category': 'Geopolitical',
 'date_publish': datetime.datetime(2018, 7, 31, 23, 45, 52),
 'year': '2018',
 'week': '31',
 'entities': {('Aetna', 'ORG'),
  ('All the Congressional Budget Office', 'ORG'),
  ('Americans', 'NORP'),
  ('Blahous', 'PERSON'),
  ('Charles', 'PERSON'),
  ('Charles Blahous', 'PERSON'),
  ('Columnist One', 'ORG'),
  ('David Koch', 'PERSON'),
  ('Democrats', 'NORP'),
  ('Facebook', 'ORG'),
  ('Gallup —', 'ORG'),
  (

as one can observe, the article's global risk information includes its **classification**: risk type, category, the similarity between the risk type embedding and the article embedding, as well as the date and the **entities** (e.g., persons, organizations, locations) identified in the article's text.

---


### Relation Extraction

<img src="images/03-b-pipeline-relation-extraction.png" width="80%" style="text-align: center;" />


For the **Relation Extraction** phase, we use the _Jaccard Similarity_ (Strategy 1 in the Figure above) since it is faster and more accurate according to our evaluation than Strategy 2 using the Semantic Similarity. Next, we illustrate the steps of the link weights computation.

In [6]:
# based on the risk information extracted in the previous phases, let's build first a dictionary risk_id -> Counter of entities
risk2entities = m.extract_risk_entities(global_risks_infos)

In [7]:
print('Example for Global Risk "Failure of climate-change mitigation and adaptation" with id "FAILURECLIMATE"')
risk2entities['FAILURECLIMATE'].most_common(10)

Example for Global Risk "Failure of climate-change mitigation and adaptation" with id "FAILURECLIMATE"


[(('Ireland', 'GPE'), 4),
 (('Northern Ireland', 'GPE'), 4),
 (('Brazil', 'GPE'), 4),
 (('India', 'GPE'), 4),
 (('the Antrim Coast', 'LOC'), 4),
 (('the Skeleton Coast', 'LOC'), 3),
 (('Chilean', 'NORP'), 3),
 (('Fethiye Sailing', 'LOC'), 3),
 (('the Causeway Coast Way', 'LOC'), 3),
 (('Dante', 'PERSON'), 3)]

In [8]:
# We then compute the Jaccard Similarity between all risk pairs:
link_weights = m.compute_link_weights_jaccard(risk2entities)

In [9]:
# For the graph of Global Risk interconnections, we focus on the mostimportant top-n links per risk:
topn_links = m.compute_topn_links(link_weights, topn=8)

In [10]:
list(topn_links.items())[0:3]

[('ASSET',
  [('INFLATION', 0.04504504504504504),
   ('INTERSTATE', 0.04054054054054054),
   ('TERRORIST', 0.01680672268907563),
   ('ENERGY', 0.013986013986013986),
   ('WEATHEREVENTS', 0.012077294685990338),
   ('ILLICITTRADE', 0.011124845488257108),
   ('FINANCIALMECH', 0.010638297872340425),
   ('WMD', 0.010309278350515464)]),
 ('CRITICALINFO',
  [('ILLICITTRADE', 0.03889943074003795),
   ('FOOD', 0.037142857142857144),
   ('STATECOLLAPSE', 0.0365296803652968),
   ('TERRORIST', 0.03592814371257485),
   ('NATIONALGOV', 0.03283302063789869),
   ('UNEMPLOYMENT', 0.0319693094629156),
   ('ENERGY', 0.028811524609843937),
   ('INFLATION', 0.02414486921529175)]),
 ('CRITICALINFRA',
  [('UNEMPLOYMENT', 0.03289473684210526),
   ('ENERGY', 0.030395136778115502),
   ('WEATHEREVENTS', 0.029880478087649404),
   ('ILLICITTRADE', 0.02346368715083799),
   ('WATER', 0.023166023166023165),
   ('FOOD', 0.020522388059701493),
   ('DATAFRAUD', 0.020361990950226245),
   ('INFLATION', 0.01851851851851851

$ $

With the information at hand, we are in the position to generate the necessary structure for the graph and express it in a json format that will help us to rendere it.

$ $

In [16]:
graph_data = m.generate_data_graph(topn_links)
print(json.dumps(graph_data, sort_keys=True, indent=4))

[
    {
        "group": 0,
        "id": 0,
        "info": {
            "category": "Economic",
            "category_idx": 0,
            "description": "Unsustainable overpriced assets such as commodities, housing, shares, etc. in a major economy or region",
            "id": "ASSET",
            "keywords": "financial crises, banking crises, banking panics, recession, economic bubble, asset bubble, speculative bubble, market bubble, price bubble, financial bubble, speculative mania, currency crises, sovereign debt, currency crises, financial sector instability, macroeconomic vulnerability, macroeconomic debt",
            "label": "Asset bubbles in a major economy",
            "long_label": "Asset bubbles in a major economy",
            "risk_idx": 0,
            "type": "Risk"
        },
        "links": [
            [
                "Economic.Unmanageable inflation",
                0.04504504504504504
            ],
            [
                "Geopolitical.Interstate co

---

# Graph Creation and Nowcasing

<img src="images/04-b-pipeline-nowcasting.png" width="80%" style="text-align: center;" />



$ $

So far, we have presented the steps to generate a Graph of Global Risk Interconnections for a given point in time $t$. If we proceed analogously for different time points $t_1, ..., t_n$, we can form a time series of graphs (e.g., link weights), that we will use to predict a future graph for the next time step(s) (i.e., nowcast).

For instance, the time series shown below represents a graph (e.g., its nodes and weighted links) per week for 2018:
$ $

In [4]:
timeseries_df = pd.read_csv('../data/2018_risk_timeseries.csv', index_col='TIME')
timeseries_df

,ASSET_BIODIVERSITY,ASSET_CRITICALINFO,ASSET_CRITICALINFRA,ASSET_CYBERATTACKS,ASSET_DATAFRAUD,ASSET_DEFLATION,ASSET_ENERGY,ASSET_FAILURECLIMATE,ASSET_FAILUREURBAN,ASSET_FINANCIALMECH,...,TERRORIST_UNEMPLOYMENT,TERRORIST_WATER,TERRORIST_WEATHEREVENTS,TERRORIST_WMD,UNEMPLOYMENT_WATER,UNEMPLOYMENT_WEATHEREVENTS,UNEMPLOYMENT_WMD,WATER_WEATHEREVENTS,WATER_WMD,WEATHEREVENTS_WMD
TIME,,,,,,,,,,,,,,,,,,,,,
2018W01,0.015514,0.027157,0.033558,0.013999,0.029515,0.017090,0.048068,0.033105,0.035499,0.078148,...,0.036117,0.045568,0.049550,0.080109,0.059139,0.072669,0.063747,0.078135,0.082461,0.076176
2018W02,0.015556,0.026634,0.036592,0.013080,0.031817,0.029630,0.055985,0.032649,0.026325,0.101493,...,0.030490,0.044768,0.050988,0.075545,0.053036,0.054084,0.056406,0.070087,0.070217,0.078474
2018W03,0.008687,0.029494,0.046430,0.013615,0.030219,0.005329,0.062821,0.033338,0.034197,0.104651,...,0.043468,0.051143,0.050438,0.076177,0.061745,0.059905,0.058747,0.075591,0.068815,0.064226
2018W04,0.014648,0.031405,0.046180,0.015570,0.034972,0.016109,0.066188,0.034884,0.026211,0.111258,...,0.038602,0.046710,0.049515,0.076196,0.055711,0.054502,0.054239,0.084215,0.059294,0.065169
2018W05,0.009817,0.029071,0.047228,0.013015,0.029245,0.015484,0.061706,0.028255,0.025067,0.116395,...,0.030958,0.042715,0.047532,0.074147,0.051825,0.049761,0.049696,0.071957,0.056423,0.063637
2018W06,0.009276,0.037251,0.056413,0.012707,0.039053,0.014403,0.067509,0.038848,0.034649,0.125179,...,0.037073,0.038945,0.046851,0.082812,0.054806,0.053476,0.053620,0.068006,0.056925,0.071523
2018W07,0.016883,0.029916,0.048963,0.015056,0.035819,0.012662,0.065042,0.043123,0.031017,0.108581,...,0.037872,0.043286,0.050201,0.076960,0.049810,0.057548,0.053357,0.069192,0.054960,0.066747
2018W08,0.011761,0.031517,0.042601,0.012817,0.028850,0.013345,0.053957,0.034286,0.027142,0.079010,...,0.041105,0.049982,0.050694,0.085047,0.057395,0.060704,0.058382,0.079902,0.062783,0.070027
2018W09,0.014706,0.026737,0.036962,0.013174,0.029748,0.011033,0.045754,0.032538,0.024622,0.076688,...,0.029068,0.046950,0.042376,0.071070,0.052463,0.065091,0.055411,0.070928,0.063720,0.059863


## For Nowcasting:
We provide an approach called _Nowcaster Network_ based on Deep Convolutional Neural Networks (CNN) based on a WaveNet architecture.



In [4]:
# let's compute and define some variables that

# the timeseries as pandas' dataframe
timeseries_df = pd.read_csv('../data/2018_risk_timeseries.csv', index_col='TIME')

# the timeseries as numpy array to be used as input to train our models
timeseries = timeseries_df.values

# the columns names
columns = timeseries_df.columns

# the window size to be used as a training instance
window_size = 4

# the time step correspond to a week. The pairs are the nodes defining a link in the graph
weeks, pairs = timeseries.shape
print('weeks: ', weeks)
print('global risks pairs (links): ', pairs)

weeks:  35
global risks pairs (links):  435


#### Simple Nowcasting Network without Residual or Skip Connections (NNS) :

<img src="images/nns.png" width="20%" style="text-align: center;" />


In [5]:
# Simple Nowcasting Network (NNS)
nns = NNS()

In [6]:
# trained with full train:
nns_model, nns_nowcast = nns.nowcast(timeseries, window_size)

X (31, 4, 435)
y (31, 435)
epochs  124
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 4, 435)            0         
_________________________________________________________________
initial_conv (Conv1D)        (None, 4, 4)              1744      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 4, 4)              84        
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 4, 4)              84        
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 4, 4)              84        
_________________________________________________________________
flatten_1 (Flatten)          (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4

In [7]:
nns_nowcast.shape

(435,)

In [8]:
nns_predicted_link_weights = [tuple(x[0].split("_") + [x[1]]) for x in list(zip(columns, nns_nowcast))]

In [9]:
nns_topn_links = m.compute_topn_links(nns_predicted_link_weights, topn=8)

In [10]:
nns_predicted_graph_data = m.generate_data_graph(nns_topn_links)
print(json.dumps(nns_predicted_graph_data, sort_keys=True, indent=4))

[
    {
        "group": 0,
        "id": 0,
        "info": {
            "category": "Economic",
            "category_idx": 0,
            "description": "Unsustainable overpriced assets such as commodities, housing, shares, etc. in a major economy or region",
            "id": "ASSET",
            "keywords": "financial crises, banking crises, banking panics, recession, economic bubble, asset bubble, speculative bubble, market bubble, price bubble, financial bubble, speculative mania, currency crises, sovereign debt, currency crises, financial sector instability, macroeconomic vulnerability, macroeconomic debt",
            "label": "Asset bubbles in a major economy",
            "long_label": "Asset bubbles in a major economy",
            "risk_idx": 0,
            "type": "Risk"
        },
        "links": [
            [
                "Economic.Unmanageable inflation",
                0.11350438743829727
            ],
            [
                "Economic.Failure of financ

---

#### Nowcasting Network with Residual and Skip Connections (NNX):

<img src="images/nnx.png" width="80%" style="text-align: center;" />


The next steps demonstrate how to predict the graph (nowcast) for the next time step using the _Nowcaster Network_ with Residual and Skip Connections (**NNX**)

In [11]:
nnx = NNX()

In [12]:
# trained with full train:
nnx_model, nnx_nowcast = nnx.nowcast(timeseries, window_size)

X (31, 4, 435)
y (31, 435)
epochs  124
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 4, 435)       0                                            
__________________________________________________________________________________________________
initial_conv (Conv1D)           (None, 4, 4)         1744        input_layer[0][0]                
__________________________________________________________________________________________________
dilated_conv_1_tanh_s0 (Conv1D) (None, 4, 4)         84          initial_conv[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 4, 4)         0           dilated_conv_1_tanh_s0[0][0]     
______________________________________________________________________

In [13]:
nnx_nowcast.shape

(435,)

In [14]:
nnx_predicted_link_weights = [tuple(x[0].split("_") + [x[1]]) for x in list(zip(columns, nnx_nowcast))]

In [15]:
nnx_topn_links = m.compute_topn_links(nnx_predicted_link_weights, topn=8)

In [16]:
nnx_predicted_graph_data = m.generate_data_graph(nnx_topn_links)
print(json.dumps(nnx_predicted_graph_data, sort_keys=True, indent=4))

[
    {
        "group": 0,
        "id": 0,
        "info": {
            "category": "Economic",
            "category_idx": 0,
            "description": "Unsustainable overpriced assets such as commodities, housing, shares, etc. in a major economy or region",
            "id": "ASSET",
            "keywords": "financial crises, banking crises, banking panics, recession, economic bubble, asset bubble, speculative bubble, market bubble, price bubble, financial bubble, speculative mania, currency crises, sovereign debt, currency crises, financial sector instability, macroeconomic vulnerability, macroeconomic debt",
            "label": "Asset bubbles in a major economy",
            "long_label": "Asset bubbles in a major economy",
            "risk_idx": 0,
            "type": "Risk"
        },
        "links": [
            [
                "Economic.Unmanageable inflation",
                0.11540775001049042
            ],
            [
                "Economic.Failure of financ

---

### Evaluation

Ground truth for the evaluation: Global Risk Graph from the WEF:

In [17]:
# ground truth Global Risk Graph links:
gt_links = dict()
for year in [2015, 2016, 2017, 2018]:
    gt_links[year] = toolbox.load_wef_ground_truth("../data/wef_ground_truth/{}/connections.tsv".format(year))

In [19]:
topn = 3

nns_metrics_mean = dict()
nnx_metrics_mean = dict()

for year in [2015, 2016, 2017, 2018]:
    nns_metrics_mean[year] = toolbox.compute_prec_rec_f1_jac_metrics(gt_links[year], nns_topn_links, topn)
    nnx_metrics_mean[year] = toolbox.compute_prec_rec_f1_jac_metrics(gt_links[year], nnx_topn_links, topn)

# ---

print()
print('nns_metrics_mean')
print(nns_metrics_mean)
print()
print('nnx_metrics_mean')
print(nnx_metrics_mean)


nns_metrics_mean
{2015: {'precision': 0.6777777777777777, 'recall': 0.11536840590863115, 'f1': 0.19717460259112274, 'jaccard': 0.11179717582039564}, 2016: {'precision': 0.788888888888889, 'recall': 0.14843675516663132, 'f1': 0.24986003017482722, 'jaccard': 0.1451666797255032}, 2017: {'precision': 0.888888888888889, 'recall': 0.12123922542275709, 'f1': 0.21337531120836675, 'jaccard': 0.11990853561730623}, 2018: {'precision': 0.9111111111111112, 'recall': 0.11844336477209515, 'f1': 0.20963449377201146, 'jaccard': 0.11725209696062273}}

nnx_metrics_mean
{2015: {'precision': 0.6777777777777777, 'recall': 0.11536840590863115, 'f1': 0.19717460259112274, 'jaccard': 0.11179717582039564}, 2016: {'precision': 0.788888888888889, 'recall': 0.14843675516663132, 'f1': 0.24986003017482722, 'jaccard': 0.1451666797255032}, 2017: {'precision': 0.9000000000000001, 'recall': 0.12290589208942376, 'f1': 0.21627659736035867, 'jaccard': 0.1215679872767579}, 2018: {'precision': 0.9, 'recall': 0.11685606318479

---

**~ Fin. ~**